In [25]:
import rosbag
import tf
from geometry_msgs import msg
import tf_conversions.posemath as posemath
import numpy as np

In [29]:
input_bag = "/home/ruffsl/git/ruffsl/my_ros_scripts/rosbag/raw_ron_data19__2015-09-14-21-01-48.bag"
output_bag = "/home/ruffsl/git/ruffsl/my_ros_scripts/rosbag/raw_ron_data19__2015-09-14-21-01-48.bag"
centroid2optic = np.matrix([[0, 0, 1, -0.271646],
                          [-1, 0, 0, 0.00771],
                          [0, -1, 0, -0.0827],
                          [0, 0, 0, 1]])

http://www.euclideanspace.com/maths/geometry/rotations/conversions/eulerToMatrix/examples/index.htm

In [31]:
with rosbag.Bag(output_bag, 'w') as outbag:
    for topic, msg, t in rosbag.Bag(input_bag).read_messages():
        # This also replaces tf timestamps under the assumption 
        # that all transforms in the message share the same timestamp
        if topic == "/Robot_1/pose":
            q = np.zeros(4)
            q[0] = msg.pose.orientation.x
            q[1] = msg.pose.orientation.y
            q[2] = msg.pose.orientation.z
            q[3] = msg.pose.orientation.w
            P = tf.transformations.quaternion_matrix(q)
            P[0, 3] = msg.pose.position.x
            P[1, 3] = msg.pose.position.y
            P[2, 3] = msg.pose.position.z
            P_ = P*centroid2optic

            q = tf.transformations.quaternion_from_matrix(P_)
            msg.pose.orientation.x = q[0]
            msg.pose.orientation.y = q[1]
            msg.pose.orientation.z = q[2]
            msg.pose.orientation.w = q[3]
            msg.pose.position.x = P_[0, 3]
            msg.pose.position.y = P_[1, 3]
            msg.pose.position.z = P_[2, 3]
#             print msg
#             print P
#             print P_
#             break
            outbag.write(topic, msg, msg.header.stamp)
        else:
            outbag.write(topic, msg, msg.header.stamp if msg._has_header else t)

In [74]:
R = tf.transformations.quaternion_matrix([0.06146124, 0, 0, 0.99810947])
R

array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.99244503, -0.12269009,  0.        ],
       [ 0.        ,  0.12269009,  0.99244503,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [28]:
# bag = rosbag.Bag(rosbag_path)
# for topic, msg, t in bag.read_messages(topics=['chatter', 'numbers']):
# #     print msg
#     print topic
# bag.close()

In [63]:
# R = numpy.matrix([[0, 0, 0],
#                   [0, 0, 0],
#                   [0, 0, 0]])

# q = np.array([0,0,1,0])

# quaternion_to_R(q, R)

# print R

# M = numpy.identity(4)
# M[:3, :3] = R
# print M

# R = numpy.identity(4)
# # quaternion_from_matrix(R, True)
# tf.transformations.quaternion_from_matrix(R)
# # q = msg.Quaternion(0,0,0,1)
# q = np.array([0,0,1,0])
# tf.transformations.quaternion_matrix(q)

In [55]:
q = msg.Quaternion(0,0,0,1)


In [62]:
# def quaternion_to_R(quat, matrix):
#     """Convert a quaternion into rotation matrix form.

#     @param quat:    The quaternion.
#     @type quat:     numpy 4D, rank-1 array
#     @param matrix:  A 3D matrix to convert to a rotation matrix.
#     @type matrix:   numpy 3D, rank-2 array
#     """

#     # Repetitive calculations.
#     q4_2 = quat[3]**2
#     q12 = quat[0] * quat[1]
#     q13 = quat[0] * quat[2]
#     q14 = quat[0] * quat[3]
#     q23 = quat[1] * quat[2]
#     q24 = quat[1] * quat[3]
#     q34 = quat[2] * quat[3]

#     # The diagonal.
#     matrix[0, 0] = 2.0 * (quat[0]**2 + q4_2) - 1.0
#     matrix[1, 1] = 2.0 * (quat[1]**2 + q4_2) - 1.0
#     matrix[2, 2] = 2.0 * (quat[2]**2 + q4_2) - 1.0

#     # Off-diagonal.
#     matrix[0, 1] = 2.0 * (q12 - q34)
#     matrix[0, 2] = 2.0 * (q13 + q24)
#     matrix[1, 2] = 2.0 * (q23 - q14)

#     matrix[1, 0] = 2.0 * (q12 + q34)
#     matrix[2, 0] = 2.0 * (q13 - q24)
#     matrix[2, 1] = 2.0 * (q23 + q14)

# def quaternion_from_matrix(matrix, isprecise=False):
#     """Return quaternion from rotation matrix.

#     If isprecise is True, the input matrix is assumed to be a precise rotation
#     matrix and a faster algorithm is used.

#     >>> q = quaternion_from_matrix(numpy.identity(4), True)
#     >>> numpy.allclose(q, [1, 0, 0, 0])
#     True
#     >>> q = quaternion_from_matrix(numpy.diag([1, -1, -1, 1]))
#     >>> numpy.allclose(q, [0, 1, 0, 0]) or numpy.allclose(q, [0, -1, 0, 0])
#     True
#     >>> R = rotation_matrix(0.123, (1, 2, 3))
#     >>> q = quaternion_from_matrix(R, True)
#     >>> numpy.allclose(q, [0.9981095, 0.0164262, 0.0328524, 0.0492786])
#     True
#     >>> R = [[-0.545, 0.797, 0.260, 0], [0.733, 0.603, -0.313, 0],
#     ...      [-0.407, 0.021, -0.913, 0], [0, 0, 0, 1]]
#     >>> q = quaternion_from_matrix(R)
#     >>> numpy.allclose(q, [0.19069, 0.43736, 0.87485, -0.083611])
#     True
#     >>> R = [[0.395, 0.362, 0.843, 0], [-0.626, 0.796, -0.056, 0],
#     ...      [-0.677, -0.498, 0.529, 0], [0, 0, 0, 1]]
#     >>> q = quaternion_from_matrix(R)
#     >>> numpy.allclose(q, [0.82336615, -0.13610694, 0.46344705, -0.29792603])
#     True
#     >>> R = random_rotation_matrix()
#     >>> q = quaternion_from_matrix(R)
#     >>> is_same_transform(R, quaternion_matrix(q))
#     True
#     >>> R = euler_matrix(0.0, 0.0, numpy.pi/2.0)
#     >>> numpy.allclose(quaternion_from_matrix(R, isprecise=False),
#     ...                quaternion_from_matrix(R, isprecise=True))
#     True

#     """
#     M = numpy.array(matrix, dtype=numpy.float64, copy=False)[:4, :4]
#     if isprecise:
#         q = numpy.empty((4, ))
#         t = numpy.trace(M)
#         if t > M[3, 3]:
#             q[0] = t
#             q[3] = M[1, 0] - M[0, 1]
#             q[2] = M[0, 2] - M[2, 0]
#             q[1] = M[2, 1] - M[1, 2]
#         else:
#             i, j, k = 1, 2, 3
#             if M[1, 1] > M[0, 0]:
#                 i, j, k = 2, 3, 1
#             if M[2, 2] > M[i, i]:
#                 i, j, k = 3, 1, 2
#             t = M[i, i] - (M[j, j] + M[k, k]) + M[3, 3]
#             q[i] = t
#             q[j] = M[i, j] + M[j, i]
#             q[k] = M[k, i] + M[i, k]
#             q[3] = M[k, j] - M[j, k]
#         q *= 0.5 / math.sqrt(t * M[3, 3])
#     else:
#         m00 = M[0, 0]
#         m01 = M[0, 1]
#         m02 = M[0, 2]
#         m10 = M[1, 0]
#         m11 = M[1, 1]
#         m12 = M[1, 2]
#         m20 = M[2, 0]
#         m21 = M[2, 1]
#         m22 = M[2, 2]
#         # symmetric matrix K
#         K = numpy.array([[m00-m11-m22, 0.0,         0.0,         0.0],
#                          [m01+m10,     m11-m00-m22, 0.0,         0.0],
#                          [m02+m20,     m12+m21,     m22-m00-m11, 0.0],
#                          [m21-m12,     m02-m20,     m10-m01,     m00+m11+m22]])
#         K /= 3.0
#         # quaternion is eigenvector of K that corresponds to largest eigenvalue
#         w, V = numpy.linalg.eigh(K)
#         q = V[[3, 0, 1, 2], numpy.argmax(w)]
#     if q[0] < 0.0:
#         numpy.negative(q, q)
#     return q